# Quantum Approximate Optimization Algorithm

In this notebook we are going to show how to use the implementation of QAOA available in Aqua to obtain solutions to the MaxCut problem

In [1]:
# since aqua was deprecated on version 0.9.0, we need to downgrade qiskit to a version that still supports it
!pip install qiskit==0.28.0

In [2]:
import numpy as np

from qiskit import Aer, IBMQ
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import *
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.providers.aer.noise import NoiseModel

provider = IBMQ.load_account()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/opt/conda/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


First, we define a function that from the coefficients of an Ising model creates the Hamiltonian for which we are going to find the ground state.

In [3]:
def get_operator(J,h,n):    
    pauli_list = []

    for (i,j) in J: # For each coefficient in J (couplings) we add a term J[i,j]Z_iZj
        x_p = np.zeros(n, dtype=np.bool)
        z_p = np.zeros(n, dtype=np.bool)
        z_p[n-1-i] = True 
        z_p[n-1-j] = True
        pauli_list.append([J[(i,j)],Pauli(z_p, x_p)])
     
    for i in h: # For each coefficient in h we add a term h[i]Z_i
        x_p = np.zeros(n, dtype=np.bool)
        z_p = np.zeros(n, dtype=np.bool)
        z_p[n-1-i] = True
        pauli_list.append([h[i],Pauli(z_p, x_p)])
    
    return WeightedPauliOperator(paulis=pauli_list)

Now, we define the edges of the graph and obtain the Hamiltonian. For this graph, which is a cycle of length 5, the optimal solution gives a cost of -3

In [4]:
# Edges of the graph

J1 = {(0,1):1, (1,2):1, (2,3):1, (3,4):1, (4,0):1}
h1 = {}
n = 5

# Hamiltonian

q_op =get_operator(J1,h1,n) 
print(q_op)
q_op.print_details()

Representation: paulis, qubits: 5, size: 5


/tmp/ipykernel_255/3746474484.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x_p = np.zeros(n, dtype=np.bool)
/tmp/ipykernel_255/3746474484.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  z_p = np.zeros(n, dtype=np.bool)
/opt/conda/lib/python3.8/site-packages/qiskit/quantum_info/operators/symplectic/pauli.py:195: DeprecationWarning: Initia

'ZZIII\t(1+0j)\nIZZII\t(1+0j)\nIIZZI\t(1+0j)\nIIIZZ\t(1+0j)\nZIIIZ\t(1+0j)\n'

We are going to run 10 repetitions on the statevector simulator

In [5]:
rep = 10
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)

/opt/conda/lib/python3.8/site-packages/qiskit/aqua/quantum_instance.py:135: DeprecationWarning: The class qiskit.aqua.QuantumInstance is deprecated. It was moved/refactored to qiskit.utils.QuantumInstance (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_class('aqua.QuantumInstance',


We run QAOA with COBYLA as the classical optimizer and with optimization level $p = 1$

In [6]:
p = 1
val = 0
for i in range(rep):
    print("----- ITERATION ",i, " ------")
    optimizer = COBYLA()
    qaoa = QAOA(q_op, optimizer, p=p)
    result = qaoa.run(quantum_instance)
    print("Optimal value", result['optimal_value'])
    val+=result['optimal_value']
print("----- AVERAGE -----")
print("Average value",val/rep)

----- ITERATION  0  ------


/opt/conda/lib/python3.8/site-packages/qiskit/aqua/components/optimizers/optimizer.py:49: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.components.optimizers',
/opt/conda/lib/python3.8/site-packages/qiskit/aqua/algorithms/vq_algorithm.py:70: DeprecationWarning: The class qiskit.aqua.algorithms.VQAlgorithm is deprecated. It was moved/refactored to qiskit.algorithms.VariationalAlgorithm (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_class('aqua.algorithms.VQAlgorithm',
/opt/conda/lib/python3.8/site-packages/qiskit/aqua/components/variational_forms/variational_form.py:48: DeprecationWarning: The package qiskit.aqua.components.variational_forms is deprecated. For mo

Optimal value -2.499999668827157
----- ITERATION  1  ------
Optimal value -2.49999973600063
----- ITERATION  2  ------
Optimal value -2.4999998774871006
----- ITERATION  3  ------
Optimal value -2.499999754026706
----- ITERATION  4  ------
Optimal value -2.4999998316924903
----- ITERATION  5  ------
Optimal value -2.4999998446497456
----- ITERATION  6  ------
Optimal value -2.499999786823413
----- ITERATION  7  ------
Optimal value -2.4999995935626256
----- ITERATION  8  ------
Optimal value -2.4999998739845095
----- ITERATION  9  ------
Optimal value -2.4999996819424597
----- AVERAGE -----
Average value -2.499999764899684


Now, we increase $p$ to $2$

In [7]:
p = 2
val = 0
for i in range(rep):
    print("----- ITERATION ",i, " ------")
    optimizer = COBYLA()
    qaoa = QAOA(q_op, optimizer, p=p)
    result = qaoa.run(quantum_instance)
    print("Optimal value", result['optimal_value'])
    val+=result['optimal_value']
print("----- AVERAGE -----")
print("Average value",val/rep)

----- ITERATION  0  ------
Optimal value -2.999985766953787
----- ITERATION  1  ------
Optimal value -2.999986421305983
----- ITERATION  2  ------
Optimal value -2.9999705827353855
----- ITERATION  3  ------
Optimal value -2.9999711074488227
----- ITERATION  4  ------
Optimal value -2.9999883805541123
----- ITERATION  5  ------
Optimal value -2.9999995526234344
----- ITERATION  6  ------
Optimal value -2.9999765342002
----- ITERATION  7  ------
Optimal value -2.9993140430504255
----- ITERATION  8  ------
Optimal value -2.9999999299588405
----- ITERATION  9  ------
Optimal value -2.9999996249356853
----- AVERAGE -----
Average value -2.9999191943766674


We are going to run the algorithm with a backend which includes a noise model

In [8]:
rep = 10
backendIBM = provider.get_backend('ibmq_belem')
noise_model = NoiseModel.from_backend(backendIBM)
coupling_map = backendIBM.configuration().coupling_map
basis_gates = noise_model.basis_gates
backend = Aer.get_backend("qasm_simulator")


shots = 8192
optimization_level = 3
p = 1
quantum_instance = QuantumInstance(backend, shots = shots, 
                                    optimization_level = optimization_level,
                                    noise_model = noise_model,
                                    basis_gates = basis_gates,
                                    coupling_map = coupling_map)

In [9]:
p = 1
val = 0
for i in range(rep):
    print("----- ITERATION ",i, " ------")
    optimizer = COBYLA()
    qaoa = QAOA(q_op, optimizer, p=p)
    result = qaoa.run(quantum_instance)
    print("Optimal value", result['optimal_value'])
    val+=result['optimal_value']
print("----- AVERAGE -----")
print("Average value",val/rep)

----- ITERATION  0  ------
Optimal value -0.7060546875
----- ITERATION  1  ------
Optimal value -0.74560546875
----- ITERATION  2  ------
Optimal value -0.7587890625
----- ITERATION  3  ------
Optimal value -0.68408203125
----- ITERATION  4  ------
Optimal value -0.70166015625
----- ITERATION  5  ------
Optimal value -0.68505859375
----- ITERATION  6  ------
Optimal value -0.76611328125
----- ITERATION  7  ------
Optimal value -0.66845703125
----- ITERATION  8  ------
Optimal value -0.7724609375
----- ITERATION  9  ------
Optimal value -0.75244140625
----- AVERAGE -----
Average value -0.724072265625
